In [245]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
df = pd.read_json('final_result_after_logistic.json')

In [92]:
df_knn = df[['asin','overall','predicted_rating','processedReview', 'summary']]

In [134]:
df_product_review_data_from_sentiment = df_knn.groupby("asin", as_index=False).mean()

In [135]:
df_product_review_data_from_sentiment.head()

,asin,overall,predicted_rating
0,0005019281,4.458716,4.940745
1,0307514161,4.732283,4.927104
2,0310263662,3.962461,4.796995
3,0310274281,4.869159,4.947654
4,0767002652,4.296875,4.842200


In [179]:
df_merge_review_series = df_knn.groupby("asin")["summary"].apply(list).reset_index()
df_merge_review_data = pd.DataFrame(df_merge_review_series)

In [180]:
df_merge_review_data.head()

,asin,summary
0,0005019281,"[good version of a classic, Good but not as mo..."
1,0307514161,[Who needs it to be christmas to watch this fl...
2,0310263662,[watching anyone being tortured would be sad -...
3,0310274281,"[great life lessons, Awesome, Inspiring, GREAT..."
4,0767002652,"[Entertaining!, Definitely a masterpiece! How..."


In [181]:
final_knn_data = pd.merge(df_product_review_data_from_sentiment, d, on="asin", how='inner')

In [182]:
final_knn_data.head()

,asin,overall,predicted_rating,summary
0,0005019281,4.458716,4.940745,"[good version of a classic, Good but not as mo..."
1,0307514161,4.732283,4.927104,[Who needs it to be christmas to watch this fl...
2,0310263662,3.962461,4.796995,[watching anyone being tortured would be sad -...
3,0310274281,4.869159,4.947654,"[great life lessons, Awesome, Inspiring, GREAT..."
4,0767002652,4.296875,4.842200,"[Entertaining!, Definitely a masterpiece! How..."


In [191]:
regEx = re.compile('[^a-z]+')
def clean_data(text_array):
    text = " ".join(text_array)
    text = text.lower()
    text = regEx.sub(' ', text).strip()
    return text

In [192]:
final_knn_data["clean_summary_data"] = final_knn_data["summary"].apply(clean_data)

# final_knn_data = pd.merge(df_product_review_data_from_sentiment, a, on="asin", how='inner')
# final_knn_data = pd.merge(df_merge_reviews,df_product_review_data_from_sentiment,on='asin')
# df_merge_reviews =df_merge_reviews.to_frame()

In [201]:
final_knn_data.head()

,asin,overall,predicted_rating,summary,clean_summary_data
0,0005019281,4.458716,4.940745,"[good version of a classic, Good but not as mo...",good version of a classic good but not as movi...
1,0307514161,4.732283,4.927104,[Who needs it to be christmas to watch this fl...,who needs it to be christmas to watch this fli...
2,0310263662,3.962461,4.796995,[watching anyone being tortured would be sad -...,watching anyone being tortured would be sad je...
3,0310274281,4.869159,4.947654,"[great life lessons, Awesome, Inspiring, GREAT...",great life lessons awesome inspiring great pic...
4,0767002652,4.296875,4.842200,"[Entertaining!, Definitely a masterpiece! How...",entertaining definitely a masterpiece how refr...


In [202]:
final_knn_data_clean = final_knn_data[['asin','overall','predicted_rating','clean_summary_data']]

In [203]:
final_knn_data_clean.head()

,asin,overall,predicted_rating,clean_summary_data
0,0005019281,4.458716,4.940745,good version of a classic good but not as movi...
1,0307514161,4.732283,4.927104,who needs it to be christmas to watch this fli...
2,0310263662,3.962461,4.796995,watching anyone being tortured would be sad je...
3,0310274281,4.869159,4.947654,great life lessons awesome inspiring great pic...
4,0767002652,4.296875,4.842200,entertaining definitely a masterpiece how refr...


In [204]:
final_knn_data_clean.to_json('final_knn_data_clean.json')


In [207]:
countVector = CountVectorizer(max_features = 300, stop_words='english') 
transformedReviews = countVector.fit_transform(final_knn_data_clean['clean_summary_data']) 

In [214]:
df_knn_vectorized_data = pd.DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
df_knn_vectorized_data = df_knn_vectorized_data.astype(int)

In [218]:
df_knn_vectorized_data.head()

,absolutely,acting,action,actors,actually,adaptation,adventure,age,amazing,american,...,world,worst,worth,worthy,wow,wrong,year,years,yes,zombie
0,0,0,0,0,0,4,0,0,0,21,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,2,1,0,...,0,0,1,0,0,0,2,0,0,0
2,2,1,1,0,1,0,0,0,10,0,...,4,4,8,1,10,1,5,2,2,0
3,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
4,1,1,1,0,0,0,0,1,0,0,...,1,0,2,1,1,0,0,2,1,0


In [221]:
final_knn_data_clean_without_summary = final_knn_data_clean[['asin','overall','predicted_rating']]


In [222]:
final_knn_data_clean_without_summary.head()

,asin,overall,predicted_rating
0,0005019281,4.458716,4.940745
1,0307514161,4.732283,4.927104
2,0310263662,3.962461,4.796995
3,0310274281,4.869159,4.947654
4,0767002652,4.296875,4.842200


In [229]:
data = final_knn_data_clean_without_summary.join(df_knn_vectorized_data, how='outer')

In [249]:
data = data.drop('asin', 1)

In [250]:
data.head()

,overall,predicted_rating,absolutely,acting,action,actors,actually,adaptation,adventure,age,...,world,worst,worth,worthy,wow,wrong,year,years,yes,zombie
0,4.458716,4.940745,0,0,0,0,0,4,0,0,...,0,0,0,1,0,0,0,0,0,0
1,4.732283,4.927104,1,0,0,0,0,0,0,2,...,0,0,1,0,0,0,2,0,0,0
2,3.962461,4.796995,2,1,1,0,1,0,0,0,...,4,4,8,1,10,1,5,2,2,0
3,4.869159,4.947654,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
4,4.296875,4.842200,1,1,1,0,0,0,0,1,...,1,0,2,1,1,0,0,2,1,0


In [251]:
np_data = np.array(data)


In [252]:
total_data = len(data)
size_of_cut = int(np.floor(0.95 * total_data))
training_data = np_data[:size_of_cut]
test_data = np_data[size_of_cut:]

In [253]:
len(training_data)

3379

In [254]:
len(test_data)

178

In [255]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(training_data)

In [256]:
distances, indices = neighbor.kneighbors(training_data)